In [2]:
%load_ext autoreload
%autoreload 2

import duckdb
import json
import os
import pandas as pd
import pathlib

# from ibge import *
from utils import *

In [4]:
# Initialize DuckDB connection
conn = duckdb.connect("Piper.db")

In [23]:
df = conn.execute("SELECT assessment_id FROM iucn").df()
df.head()

,assessment_id
0,9569685
1,9956892
2,10245715
3,10245845
4,10847879


In [ ]:
conn.execute("ALTER TABLE iucn ADD COLUMN criteria TEXT")
conn.execute("ALTER TABLE iucn ADD COLUMN presence BOOLEAN DEFAULT FALSE")
conn.execute("ALTER TABLE iucn ADD COLUMN land_regions TEXT")
conn.execute("SELECT * FROM iucn LIMIT 5").df()

,year_published,latest,possibly_extinct,possibly_extinct_in_the_wild,sis_taxon_id,url,taxon_scientific_name,red_list_category_code,assessment_id,scopes,criteria,presence,land_regions
0,1998,True,False,False,30646,https://www.iucnredlist.org/species/30646/9569685,Piper distigmatum,DD,9569685,"[{'code': '1', 'description': {'en': 'Global'}}]",None,False,None
1,1998,True,False,False,35768,https://www.iucnredlist.org/species/35768/9956892,Piper verrucosum,LR/nt,9956892,"[{'code': '1', 'description': {'en': 'Global'}}]",None,False,None
2,2004,True,False,False,39532,https://www.iucnredlist.org/species/39532/1024...,Peperomia kamerunana,EN,10245715,"[{'code': '1', 'description': {'en': 'Global'}}]",B2ab(iii),False,None
3,2000,True,False,False,39533,https://www.iucnredlist.org/species/39533/1024...,Peperomia thomeana,LR/nt,10245845,"[{'code': '1', 'description': {'en': 'Global'}}]",None,False,None
4,2003,True,False,False,44006,https://www.iucnredlist.org/species/44006/1084...,Piper seychellarum,VU,10847879,"[{'code': '1', 'description': {'en': 'Global'}}]",D2,False,None


In [35]:
conn.execute(f"""
    SELECT COUNT(*) FROM iucn WHERE taxon_scientific_name LIKE 'Piper%'
""").df()

,count_star()
0,173


In [36]:
conn.execute(f"""
    SELECT COUNT(*)
             FROM iucn
             WHERE taxon_scientific_name LIKE 'Piper%' AND
             (red_list_category_code='CR' OR red_list_category_code='EN' OR red_list_category_code='VU' OR
             red_list_category_code='NT' OR red_list_category_code='LR/nt' OR red_list_category_code='LC')
""").df()

,count_star()
0,157


In [38]:
for _, row in df.iterrows():
    url = "https://api.iucnredlist.org/api/v4/assessment/%s" % row["assessment_id"]
    print("requests %s assessment %s" % (url, row["assessment_id"]))
    headers ={"Authorization": os.environ["IUCN_API"]}
    response = make_requests(url, headers=headers)
    if response and "criteria" in response:
        conn.execute(f"""
            UPDATE iucn SET criteria = '{response["criteria"]}'
            WHERE assessment_id = '{row["assessment_id"]}'
        """)
    

requests https://api.iucnredlist.org/api/v4/assessment/9569685 assessment 9569685
requests https://api.iucnredlist.org/api/v4/assessment/9956892 assessment 9956892
requests https://api.iucnredlist.org/api/v4/assessment/10245715 assessment 10245715
requests https://api.iucnredlist.org/api/v4/assessment/10245845 assessment 10245845
requests https://api.iucnredlist.org/api/v4/assessment/10847879 assessment 10847879
requests https://api.iucnredlist.org/api/v4/assessment/11010795 assessment 11010795
requests https://api.iucnredlist.org/api/v4/assessment/11010995 assessment 11010995
requests https://api.iucnredlist.org/api/v4/assessment/11011085 assessment 11011085
requests https://api.iucnredlist.org/api/v4/assessment/11011174 assessment 11011174
requests https://api.iucnredlist.org/api/v4/assessment/11011268 assessment 11011268
requests https://api.iucnredlist.org/api/v4/assessment/11011358 assessment 11011358
requests https://api.iucnredlist.org/api/v4/assessment/11011454 assessment 11011

In [18]:
conn.execute(""" DROP TABLE IF EXISTS iucn_complete;
             CREATE TABLE iucn_complete AS 
                SELECT *
                FROM iucn
             WHERE taxon_scientific_name LIKE 'Piper%' AND
             (red_list_category_code='CR' OR red_list_category_code='EN' OR red_list_category_code='VU' OR
             red_list_category_code='NT' OR red_list_category_code='LR/nt' OR red_list_category_code='LC')""")
conn.execute("SELECT * FROM iucn_complete LIMIT 5").df()

,year_published,latest,possibly_extinct,possibly_extinct_in_the_wild,sis_taxon_id,url,taxon_scientific_name,red_list_category_code,assessment_id,scopes,criteria,presence,land_regions
0,2004,True,False,False,45864,https://www.iucnredlist.org/species/45864/1101...,Piper stipulosum,CR,11015737,"[{'code': '1', 'description': {'en': 'Global'}}]",B1ab(iii),False,None
1,2004,True,False,False,45815,https://www.iucnredlist.org/species/45815/1101...,Piper angamarcanum,CR,11016972,"[{'code': '1', 'description': {'en': 'Global'}}]",B1ab(iii),False,None
2,2004,True,False,False,45822,https://www.iucnredlist.org/species/45822/1101...,Piper bullatifolium,CR,11017903,"[{'code': '1', 'description': {'en': 'Global'}}]",B1ab(iii),False,None
3,2004,True,False,False,45826,https://www.iucnredlist.org/species/45826/1101...,Piper clathratum,CR,11018409,"[{'code': '1', 'description': {'en': 'Global'}}]",B1ab(iii),False,None
4,2004,True,False,False,45833,https://www.iucnredlist.org/species/45833/1101...,Piper entradense,CR,11019238,"[{'code': '1', 'description': {'en': 'Global'}}]",A4c,False,None


In [19]:
conn.execute("SELECT COUNT(*) FROM iucn_complete").df()

,count_star()
0,157


In [24]:
for _, row in df.iterrows():
    url = "https://api.iucnredlist.org/api/v4/assessment/%s" % row["assessment_id"]
    print("requests %s assessment %s" % (url, row["assessment_id"]))
    headers ={"Authorization": os.environ["IUCN_API"]}
    response = make_requests(url, headers=headers)
    # print(response)
    if response and "locations" in response:
        df2 = conn.execute(f"""SELECT * FROM iucn WHERE assessment_id = '{row["assessment_id"]}'""").df()
        for l in response["locations"]:
            presence = True if l["presence"] == 'Extant' else False
            land_regions = l["description"]["en"].replace("'", "''")
            print(presence, land_regions)
            sql = f"""
                INSERT INTO iucn_complete (year_published, latest, possibly_extinct, possibly_extinct_in_the_wild, sis_taxon_id, url, taxon_scientific_name, red_list_category_code, assessment_id, scopes, criteria, presence, land_regions)
         VALUES ('{df2['year_published'][0]}', {df2['latest'][0]}, {df2['possibly_extinct'][0]}, {df2['possibly_extinct_in_the_wild'][0]}, {df2['sis_taxon_id'][0]}, '{df2['url'][0]}', '{df2['taxon_scientific_name'][0]}', '{df2['red_list_category_code'][0]}', {df2['assessment_id'][0]}, {df2['scopes'][0]}, '{df2['criteria'][0]}', {presence}, '{land_regions}')                
            """
            conn.execute(sql)

requests https://api.iucnredlist.org/api/v4/assessment/9569685 assessment 9569685
True Panama
requests https://api.iucnredlist.org/api/v4/assessment/9956892 assessment 9956892
True Jamaica
requests https://api.iucnredlist.org/api/v4/assessment/10245715 assessment 10245715
True Cameroon
True Bioko
True Equatorial Guinea
requests https://api.iucnredlist.org/api/v4/assessment/10245845 assessment 10245845
True Cameroon
True Bioko
True Equatorial Guinea
True Sao Tome and Principe
requests https://api.iucnredlist.org/api/v4/assessment/10847879 assessment 10847879
True Seychelles
requests https://api.iucnredlist.org/api/v4/assessment/11010795 assessment 11010795
True Ecuador
requests https://api.iucnredlist.org/api/v4/assessment/11010995 assessment 11010995
True Ecuador
requests https://api.iucnredlist.org/api/v4/assessment/11011085 assessment 11011085
True Ecuador
requests https://api.iucnredlist.org/api/v4/assessment/11011174 assessment 11011174
True Ecuador
requests https://api.iucnredlist

In [25]:
conn.execute("SELECT DISTINCT(land_regions) FROM iucn_complete").df()

,land_regions
0,"Venezuela, Bolivarian Republic of"
1,Colombia (mainland)
2,India
3,Haiti
4,"Bolivia, Plurinational States of"
...,...
68,"Bonaire, Sint Eustatius and Saba"
69,Kerala
70,"Virgin Islands, British"
71,Dominica


In [ ]:
conn.execute(f"""
    SELECT COUNT(DISTINCT(taxon_scientific_name))
    FROM iucn_complete
    WHERE (land_regions='Uruguay') AND presence=TRUE AND taxon_scientific_name LIKE 'Piper %'
""").df()

,count(DISTINCT taxon_scientific_name)
0,0


In [37]:
conn.execute("SELECT COUNT(*) FROM iucn_complete WHERE land_regions='Venezuela, Bolivarian Republic of'").df()

,count_star()
0,30
